# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [2]:
import datasets
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random

In [3]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval, variation_count,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation_count = variation_count
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = 0
        self.labels = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, variation_count={}, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),self.variation_count,len(self.centers),self.fitness,
                      self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [4]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [5]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [6]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2


In [7]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [8]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [9]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [10]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [11]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


# Main Algorithm 

**Extra details will be explain in other next sections**
First we create objects with random centers in interval which that will selects from dataset <br>
For example in wine we have 13 features and if we need 3 clusters then our interval will be 3x13

In [12]:
candidates_array = []

''' what are u doing exactly here? '''

#for c in range(0,5):
#    candidate_array.append(Candidate(random.sample(list(wine()[0]),3),np.array(random.sample(list(wine()[0]),3)),15,-1)) 
    
#print(candidate_array[0].sampling_interval[0][10])

' what are u doing exactly here? '

## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [13]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500 # is this NFE in the article??? see table 2
variations_count = 15
number_of_clusters = 3 # in this article number of clusters are predefined

In [16]:
# initializing candidates

for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_iris,sampling_interval = sampling_intervals(x_iris),variation_count = variations_count))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [17]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #150 nodes, variation_count=15, #0 centers, fitness=-1
 clusters:
[[6.2 2.2 4.5 1.5]
 [4.6 3.4 1.4 0.3]
 [6.3 2.9 5.6 1.8]
 [5.9 3.  4.2 1.5]
 [5.8 2.7 5.1 1.9]] 
 Centers:
[]
 sampling_interval:
(array([7.9, 4.4, 6.9, 2.5]), array([4.3, 2. , 1. , 0.1]))


In [18]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[6.5 3.2 5.1 2. ]
 [5.9 3.  4.2 1.5]
 [7.1 3.  5.9 2.1]]
[[6.4 3.2 4.5 1.5]
 [4.5 2.3 1.3 0.3]
 [7.6 3.  6.6 2.1]]
[[6.8 3.2 5.9 2.3]
 [7.7 3.8 6.7 2.2]
 [6.3 2.5 5.  1.9]]
[[6.6 3.  4.4 1.4]
 [5.7 2.5 5.  2. ]
 [5.7 2.9 4.2 1.3]]
[[5.1 3.8 1.9 0.4]
 [6.5 2.8 4.6 1.5]
 [6.  2.9 4.5 1.5]]


## Sampling Interval
this function use for making sampling_interval


In [15]:

def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [21]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [22]:
candidates_array[1].fitness

78.85144142614601

## K-means Algorithm 
this function fitting k-means model on dataset

In [20]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    y_predict_kmeans = kmeans.fit_predict(x)
    return (y_predict_kmeans,kmeans.cluster_centers_,kmeans.inertia_)


## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [72]:
import copy
def mutation(candidate_array,mutation_random):
    
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        print(Trial_candidate[x].centers)
        print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        print(each_fitness(Trial_candidate[x]))
        print(each_fitness(candidate_array[x]))
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen

[[6.37983107 2.91508813 4.7482395  1.63308243]
 [4.81519914 3.39581762 0.28800415 0.0446268 ]
 [6.85       3.07368421 5.74210526 2.07105263]]
[[5.9016129  2.7483871  4.39354839 1.43387097]
 [5.006      3.428      1.462      0.246     ]
 [6.85       3.07368421 5.74210526 2.07105263]]
###########################################################################################################
[[148.06742825  50.51055092  65.11229295  97.07502616]
 [ 72.119627    47.01606524 157.72941956 201.38491891]
 [182.37        48.74462604 109.52232687  78.96479224]]
[[118.62003122  53.01745057  55.00308012 106.83570239]
 [ 78.235632    46.980208    87.054928   184.809392  ]
 [182.37        48.74462604 109.52232687  78.96479224]]
###########################################################################################################
[[5.01065187 2.61784865 3.11975699 1.27708833]
 [5.83550428 3.61599872 1.57981677 0.43528421]
 [6.85       3.07368421 5.74210526 2.07105263]]
[[5.9016129  2.7483871  4.

## Fitness over Mutated Candidate
Just simple sum of square of distances

In [24]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

In [70]:
def center_dist(data, center, labels,center_position):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l][center_position]
        resid = data_l - center
        sqe += (resid**2).sum()
    return sqe

# probability Equation

In [ ]:
def probability(inertia_array,i):
    return (1/inertia_array[i])/(1/(np.sum(inertia_array)))

In [71]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = [[0 for x in range(len(candidates_array_example.centers[0]))]for y in range(len(candidates_array_example.centers))]
    for i in range(0,len(candidates_array_example.centers)):
        for j in range(0,len(candidates_array_example.centers[i])):
            fitness_of_each_centers[i][j] = center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))
print(candidates_array[0].centers)
print(len(candidates_array[0].centers))
print(len(candidates_array[0].centers[i]))


[[118.62003122  53.01745057  55.00308012 106.83570239]
 [ 78.235632    46.980208    87.054928   184.809392  ]
 [182.37        48.74462604 109.52232687  78.96479224]]
[[5.9016129  2.7483871  4.39354839 1.43387097]
 [5.006      3.428      1.462      0.246     ]
 [6.85       3.07368421 5.74210526 2.07105263]]
3
4
